In [30]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import re

In [15]:
raw_data = pd.read_excel('data/Tonsillectomy_OUTCOME_Data_KQ5_Master.xlsx', 
              sheetname='Outcome data', 
              na_values=['null', 'ND']).drop(['Comments', 
                                              'Comments 2', 
                                              'Other stats \nName',
                                             'Other Stats',
                                             'Results'], axis=1)
raw_data.shape

(401, 42)

In [64]:
# Lower case
groups = raw_data.Group_Desc.str.lower()
# Strip information after commas
groups = groups.str.split(',').apply(lambda s: s[0])
# Remove chunks with numeric characters or 'kg', as these are dosages
groups = groups.str.split(' ').apply(lambda s: 
                        ' '.join([si for si in s 
                                  if not re.compile('[\d()/]').search(si)]))
# Combine saline, control and placebo, assume groups starting with 'no' means placebo
groups = groups.replace({'saline':'placebo',
                        'control':'placebo'}).apply(lambda s: 'placebo'*s.startswith('no ') or s)

In [65]:
groups.value_counts()

placebo                                                                 90
dexamethasone                                                           87
ramosetron                                                              27
granisetron                                                              9
ondansetron                                                              9
ibuprofen                                                                8
tropisetron                                                              7
metoclopramide                                                           7
postoperative ketoprofen + saline                                        6
levobupivacaine with epinephrine                                         6
droperidol                                                               6
dexamethasone sodium phosphate                                           6
dolasetron                                                               6
diclofenac               